In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from src.sys_prompt import mc_prompt_emotion_v2, mc_prompt_commitment_v2, mc_prompt_purpose_v2, mc_prompt_how_to_raise_funds_v1
from src.data_processing import get_data
import re
import json
import pandas as pd
import logging
import datetime

# Set file names

# Path of the processed file
file_to_save_processed = r'/Users/harshgarg/Desktop/mc-lm-genai/data/LM Applications 20250502_processed.xlsx'
# Path of the unprocessed file
file_to_import_unprocessed = r"/Users/harshgarg/Desktop/mc-lm-genai/data/LM Applications 20250502.xlsx"
# Path of partially processed file
# file_to_import_partiallyProcessed = r"/Users/harshgarg/Desktop/mc-lm-genai/data/LM Applications 20250502.xlsx"
# Path of the file to run a single row from
# file_to_import_single_cell = r"/Users/harshgarg/Desktop/mc-lm-genai/data/LM Applications 20250502.xlsx"


In [2]:
# To generate IDs (to better query logs; irrelevant otherwise)

df = get_data(path=file_to_import_unprocessed)
df.to_excel(file_to_import_unprocessed, index=False)

## Always run the below cell

In [ ]:
# Configure logging with timestamp
log_filename = f"mc_scoring_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    # handlers=[
    #     logging.FileHandler(log_filename),
    #     logging.StreamHandler()
    # ]
    filename=log_filename,
    force=True
)

# Log the start of the script
logging.info("Script execution started")


load_dotenv()
endpoint = os.getenv("ENDPOINT_URL")
deployment = os.getenv("DEPLOYMENT_NAME_1")
endpoint = os.getenv("ENDPOINT_URL")
subscription_key = os.getenv("AZURE_OPEN_AI_API_KEY")

# Initialize Azure OpenAI Service client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

df = get_data(path=file_to_import_unprocessed)

df['emotion'] = 0
df['purpose'] = 0
df['commitment'] = 0
df['fundraising'] = 0

## Helper functions (the next 2 cells)

In [4]:
def get_llm_output(prompt, row_data):
    # Log the start of the function call
    logging.info(f"Processing row data using prompt: {prompt[:50]}...")
    logging.info(f"Processing row: {row_data}")
    
    try:
        # Prepare the chat prompt
        chat_prompt = [{"role": "user", "content": [{"type": "text", "text": prompt + "\n" + str(row_data)}]}]
        messages = chat_prompt

        # Generate the completion
        logging.info("Sending request to OpenAI API")
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
        )

        # Parse the JSON content from the completion
        response_content = completion.choices[0].message.content
        logging.info("Received response")
        logging.info(f"Response: {response_content}")

        
        # Extract the JSON part from the markdown code block
        json_match = re.search(r'```(?:json)?\s*(.*?)\s*```', response_content, re.DOTALL)
        
        if json_match:
            json_str = json_match.group(1)
            result = json.loads(json_str)
            logging.info("Successfully parsed JSON response")
        else:
            # Fallback if regex doesn't match
            # Try to evaluate directly (be cautious with eval)
            logging.warning("JSON regex match failed, attempting direct eval")
            result = eval(response_content)
            
        return result
    
    except Exception as e:
        # Log any exceptions that occur
        logging.error(f"Error processing: {str(e)}")
        # Return a default value in case of error
        return {"error": str(e), "score": 0}

In [ ]:
def process_row(row_data, value, index):
    # Process fundraising scoring
    if pd.notna(row_data['spon_pledge_how_to_reach_target']) and len(str(row_data['spon_pledge_how_to_reach_target'])) > 0:
        logging.info(f"Processing fundraising for ID: {value['id']}")
        fundraising_result = get_llm_output(mc_prompt_how_to_raise_funds_v1, {'spon_pledge_how_to_reach_target': row_data['spon_pledge_how_to_reach_target']})
        logging.info(f"Fundraising score for ID {value['id']}: {fundraising_result.get('fundraising_score', 0)}")
    else:
        logging.info(f"No fundraising data for ID: {value['id']}, assigning default score 0")
        fundraising_result = {"fundraising_score": 0}


    # Process reason to run data for emotion, purpose and commitment scoring
    if pd.notna(row_data['why_run_for_marie_curie']) and len(str(row_data['why_run_for_marie_curie'])) > 0:
        logging.info(f"Processing emotional content for ID: {value['id']}")
        emotion_result = get_llm_output(mc_prompt_emotion_v2, {"reason": row_data['reason_free_text'] + row_data['reason_drop_down'] + row_data['why_run_for_marie_curie']})
        logging.info(f"Emotion score for ID {value['id']}: {emotion_result.get('emotion', 0)}")
        
        logging.info(f"Processing purpose content for ID: {value['id']}")
        purpose_result = get_llm_output(mc_prompt_purpose_v2, {"reason": row_data['reason_free_text'] + row_data['reason_drop_down'] + row_data['why_run_for_marie_curie']})
        logging.info(f"Purpose score for ID {value['id']}: {purpose_result.get('purpose', 0)}")
        
        logging.info(f"Processing commitment content for ID: {value['id']}")
        commitment_result = get_llm_output(mc_prompt_commitment_v2, {"reason": row_data['reason_free_text'] + row_data['reason_drop_down'] + row_data['why_run_for_marie_curie']})
        logging.info(f"Commitment score for ID {value['id']}: {commitment_result.get('commitment', commitment_result.get('committment', 0))}")
    else:
        logging.info(f"No reason_free_text data for ID: {value['id']}, assigning default scores 0")
        emotion_result = {"emotion": 0}
        purpose_result = {"purpose": 0}
        commitment_result = {"commitment": 0}

    # Update the dataframe with scores
    try:
        df.at[index, 'fundraising'] = fundraising_result['fundraising_score']
        df.at[index, 'emotion'] = emotion_result['emotion']
        df.at[index, 'purpose'] = purpose_result['purpose']
        
        # Handle the commitment/committment spelling variation in the result
        if 'commitment' in commitment_result:
            df.at[index, 'commitment'] = commitment_result['commitment']
        elif 'committment' in commitment_result:
            df.at[index, 'commitment'] = commitment_result['committment']
        else:
            df.at[index, 'commitment'] = 0
            logging.warning(f"No commitment score found for ID {value['id']}")
            
        # Calculate final score
        df.at[index, 'Final_score'] = 0  # Reset the score before adding components
        
        # Add scores from LLM evaluations
        df.at[index, 'Final_score'] += float(df.at[index, 'emotion']) + float(df.at[index, 'purpose']) + float(df.at[index, 'commitment']) + float(df.at[index, 'fundraising'])
        logging.info(f"Base scores sum for ID {value['id']}: {df.at[index, 'Final_score']}")
        
# Score for In Memory
        if value['in_mem'].startswith("Y"):
            df.at[index, 'Final_score'] += 1
            logging.info(f"Added 1 point for in_mem (Y) for ID {value['id']}")
        elif value['in_mem'].startswith("N"):
            df.at[index, 'Final_score'] += 0
            logging.info(f"Added 0 points for in_mem (N) for ID {value['id']}")
        else:
            df.at[index, 'Final_score'] += 0.5
            logging.info(f"Added 0.5 points for in_mem (Other) for ID {value['id']}")

        # Score for In Memory Details - in_memory_drop_down and in_memory_free_text (Currently just checking if they are populated)
        if pd.notna(row_data['in_memory_drop_down']) and len(str(row_data['in_memory_drop_down'])) > 0:
            df.at[index, 'Final_score'] += 1
            logging.info(f"Added 1 point for in_memory_drop_down (Y) for ID {value['id']}")
        else:
            df.at[index, 'Final_score'] += 0
            logging.info(f"Added 0 points for in_memory_drop_down (Blank) for ID {value['id']}")

        if pd.notna(row_data['in_memory_free_text']) and len(str(row_data['in_memory_free_text'])) > 0:
            df.at[index, 'Final_score'] += 1
            logging.info(f"Added 1 point for in_memory_free_text (Y) for ID {value['id']}")
        else:
            df.at[index, 'Final_score'] += 0
            logging.info(f"Added 0 points for in_memory_free_text (Blank) for ID {value['id']}")
        

        # Score for run before
        if value['run_LM_before'].__contains__("No"):
            df.at[index, 'Final_score'] += 0
            logging.info(f"Added 0 point for run_LM_before (No) for ID {value['id']}")
        else:
            df.at[index, 'Final_score'] += 1
            logging.info(f"Added 1 points for run_LM_before (Yes) for ID {value['id']}")

        # Score for sponsor pledge
        if value['spon_pledge_amount'] == "£0 - £2,4999":
            df.at[index, 'Final_score'] += 1
            logging.info(f"Added 1 point for sponsor pledge amount for {value['id']}: {value['spon_pledge_amount']}")
        elif value['spon_pledge_amount'] == "2500":
            df.at[index, 'Final_score'] += 2
            logging.info(f"Added 2 points for sponsor pledge amount for {value['id']}: {value['spon_pledge_amount']}")
        elif value['spon_pledge_amount'] == "£2,500 - £3,499":
            df.at[index, 'Final_score'] += 3
            logging.info(f"Added 3 points for sponsor pledge amount for {value['id']}: {value['spon_pledge_amount']}")
        elif value['spon_pledge_amount'] == "£3,500 - £3,999":
            df.at[index, 'Final_score'] += 4
            logging.info(f"Added 4 points for sponsor pledge amount for {value['id']}: {value['spon_pledge_amount']}")
        elif value['spon_pledge_amount'] == "£4,000 +":
            df.at[index, 'Final_score'] += 5
            logging.info(f"Added 5 points for sponsor pledge amount for {value['id']}: {value['spon_pledge_amount']}")
        else:
            df.at[index, 'Final_score'] += 0
            logging.info(f"Added 0 point for sponsor pledge amount for {value['id']}: Blank")

        # Score for Matched Fundraising
        if value['employee_details_matched_fundraising'].endswith("Yes"):
            df.at[index, 'Final_score'] += 1
            logging.info(f"Added 1 point for employee_details_matched_fundraising (Yes) for ID {value['id']}")
        elif value['employee_details_matched_fundraising'].endswith("No"):
            df.at[index, 'Final_score'] += 0
            logging.info(f"Added 0 points for employee_details_matched_fundraising (No) for ID {value['id']}")
        else:
            df.at[index, 'Final_score'] += 0.5
            logging.info(f"Added 0.5 points for employee_details_matched_fundraising (Unsure) for ID {value['id']}")
        
        # Score for PR_and_media_special_plans
        if value['special_plans_and/or_fancy_dress_drop_down'].startswith("No"):
            df.at[index, 'Final_score'] += 0
            logging.info(f"Added 0 point for special_plans_and/or_fancy_dress_drop_down (No) for ID {value['id']}")
        else:
            df.at[index, 'Final_score'] += 1
            logging.info(f"Added 1 points for special_plans_and/or_fancy_dress_drop_down (Some plan) for ID {value['id']}")
        
        
        # # Score for Public Ballot
        # if value['entered_public_ballot'].endswith("Yes"):
        #     df.at[index, 'Final_score'] += 0
        #     logging.info(f"Added 1 point for entered_public_ballot (Yes) for ID {value['id']}")
        # elif value['entered_public_ballot'].endswith("No"):
        #     df.at[index, 'Final_score'] += 1
        #     logging.info(f"Added 0 points for entered_public_ballot (No) for ID {value['id']}")
        # else:
        #     df.at[index, 'Final_score'] += 0.5
        #     logging.info(f"Added 0.5 points for entered_public_ballot (Other) for ID {value['id']}")
        
        # Score for Pledge Amount (normalized)
        # pledge_score = df.at[index, 'Pledge Amount']/max_amount if pd.notna(df.at[index, 'Pledge Amount']) else 0
        # df.at[index, 'Final_score'] += pledge_score
        # logging.info(f"Added {pledge_score:.4f} points for Pledge Amount for ID {value['id']}")
        
        # logging.info(f"Final score for ID {value['id']}: {df.at[index, 'Final_score']}")
        
    except Exception as e:
        logging.error(f"Error updating scores for ID {value['id']}: {str(e)}")
    
    # Log progress every 10 applications
    if index % 10 == 0 and index > 0:
        logging.info(f"Progress: {index}/{len(df)} applications processed ({index/len(df)*100:.1f}%)")

## Process a fresh (unprocessed file)

In [21]:

# Import data
df = get_data(path=file_to_import_unprocessed)

logging.info(f"Processing {len(df) if 'df' in locals() else 0} applications")

df["emotion"] = ""
df["purpose"] = ""
df["commitment"] = ""
df["fundraising"] = ""
df['Final_score'] = 0

# Pledge amount is a range in this sheet; need to incorporate processing of this
# max_amount = max(df['Pledge Amount'])

# No matched fundraising in this sheet?
# df['Matched Fundraising'] = df['Matched Fundraising'].fillna("No")
df['in_mem'] = df['in_mem'].fillna("N")
df['entered_public_ballot'] = df['entered_public_ballot'].fillna("No")

for index, value in df.iterrows():
    logging.info(f"Processing application {index+1}/{len(df)}, ID: {value['id']}")
    
    row_data = value.to_dict()
    
    process_row(row_data, value, index)

    # adding 2 to index to indicate the row number in excel
    print(f"Row {index + 2} processed.")

# Save to excel

df.to_excel(file_to_save_processed, index=False)

Row 2 processed.


/var/folders/90/vqz6m1bx3hb3j40plckgml5w0000gn/T/ipykernel_56441/2724725548.py:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Final_score'] += 0.5


Row 3 processed.
Row 4 processed.
Row 5 processed.
Row 6 processed.
Row 7 processed.
Row 8 processed.
Row 9 processed.
Row 10 processed.
Row 11 processed.
Row 12 processed.
Row 13 processed.
Row 14 processed.
Row 15 processed.
Row 16 processed.
Row 17 processed.
Row 18 processed.
Row 19 processed.
Row 20 processed.
Row 21 processed.
Row 22 processed.
Row 23 processed.
Row 24 processed.
Row 25 processed.
Row 26 processed.


KeyboardInterrupt: 

In [22]:
df.head(15)

,event_number,booking_number,start_date,event_desc,option_desc,reason,reason_notes,recruitment_source,recruitment_source_notes,registration_method,...,entered_public_ballot,supported_by_us_run_for_MC_if_gained_own_place,supported_by_us_run_for_MC_notes_yes,est_finish_time,id,emotion,purpose,commitment,fundraising,Final_score
0,11259,100707576,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",Because I work for a corporate partner,"Other, see notes",Marie Curie website,Website,...,No,NaN,NaN,NaN,76WV33RQ,0.5,0.5,0.5,0.5,6.0
1,11259,100707578,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",Because Marie Curie is the TCS London Marathon...,"Other, see notes",That is classified,Website,...,No,NaN,NaN,NaN,PJ716CD3,0.5,0.5,1.0,0,6.5
2,11259,100707579,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",Marie Curie provides/provided care for a loved...,"Other, see notes",Word of mouth,Website,...,No,NaN,NaN,NaN,P97Q9WD9,4.5,4.5,4.5,0,13.5
3,11259,100707660,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",I have a relative who received end of life car...,"Other, see notes",Marie Curie website,Website,...,No,NaN,NaN,NaN,62618400,4.8,5.0,5.0,4.5,19.8
4,11259,100707662,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",Marie Curie provides/provided care for a loved...,"Other, see notes",your nurses help a family member,Website,...,No,NaN,NaN,NaN,8E9278Q9,4.2,4.5,4.5,3.5,19.7
5,11259,100707674,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",Marie Curie provides/provided care for a loved...,"Other, see notes",Marie Curie website,Website,...,No,NaN,NaN,NaN,1K77F17H,5.0,5,5,4.0,21.0
6,11259,100707680,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",Marie Curie provides/provided care for a loved...,"Other, see notes",Marie Curie website,Website,...,No,NaN,NaN,NaN,4N7OF7QF,5.0,5.0,5,4.5,21.5
7,11259,100707712,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",I like to help people / make a difference,"Other, see notes",Marie Curie website,Website,...,No,NaN,NaN,NaN,49J44U0Y,4.8,4.7,4.7,4,20.7
8,11259,100707732,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",I have ran Dublin Marathon twice for Marie Cur...,"Other, see notes",Email from Marie Curie,Website,...,No,NaN,NaN,NaN,EBP7CX55,4.8,4.8,5,4.5,21.6
9,11259,100707736,00:00:00,TCS London Marathon 2026,Guaranteed Place - Interested,"Other, specify details in activity notes",Marie Curie provides/provided care for a loved...,"Other, see notes",They help my granny when she has cancer,Website,...,No,NaN,NaN,NaN,88YB3443,5.0,5.0,5.0,3.5,21.0
